In [ ]:
import numpy as np
import os, shutil, json, subprocess

from tqdm import tqdm
from PIL import Image
Image.MAX_IMAGE_PIXELS = None

In [ ]:
def charger_image(chemin_image, coordonnées=None):
    with Image.open(chemin_image) as img:
        return img.crop(coordonnées) if coordonnées else img.copy()

def lister_dossiers(chemin_dossier_parent="./", commencent_par=None):
    return [os.path.join(chemin_dossier_parent, dossier) for dossier in os.listdir(chemin_dossier_parent) if os.path.isdir(os.path.join(chemin_dossier_parent, dossier)) and dossier.startswith(commencent_par)]

def lister_fichiers(chemin_dossier_parent="./"):
    return [fichier for fichier in os.listdir(chemin_dossier_parent) if os.path.isfile(os.path.join(chemin_dossier_parent, fichier))]

def supprimer_dossiers(dossiers):
    for dossier in dossiers:
        if os.path.exists(dossier):
            shutil.rmtree(dossier)
    
def créer_dossiers(dossiers):
    for dossier in dossiers:
        os.makedirs(dossier, exist_ok=True)

def couper_image(image, largeur, longueur, dimension):
    return image.crop((0, 0, (largeur // dimension) * dimension, (longueur // dimension) * dimension))

def est_noire(image, seuil=40):
    image_array = np.array(image)
    image_array_rgb = image_array[..., :3]
    return ((np.sum(np.all(image_array_rgb < [3, 3, 3], axis=-1)) / (image_array_rgb.shape[0] * image_array_rgb.shape[1])) * 100) > seuil

def est_obscure(image, seuil=25):
    return np.mean(np.array(image), axis=(0, 1)).max() < seuil

def est_grise(image):
    return all(abs(rouge - vert) <= 3 and abs(vert - bleu) <= 3 for rouge, vert, bleu in image.convert("RGB").getdata())

def charger_dictionnaire(chemin_fichier):
    with open(chemin_fichier, "r", encoding="utf-8") as fichier:
        return json.load(fichier)

def sauvegarder_dictionnaire(chemin_fichier, dictionnaire):
    with open(chemin_fichier, "w") as fichier:
        json.dump(dictionnaire, fichier, indent=4)

In [ ]:
zones = charger_dictionnaire("../données/zones.json")
zones = [zone for zone in zones.values() if zone.get("type") == "localisation"]

for zone in zones:

    coordonnées = {}
    carte = charger_image(f"../images/cartes/{zone['id']}/corrigée_petite.webp")
    screenshot = charger_image(f"../images/screenshots/{zone['id']}/screenshot.png")
    
    print(f"Zone '{zone['id']}' :")
    supprimer_dossiers([f"../données/{zone['id']}"])
                
    for dimension in zone["dimensions"]["associées_aux_difficultés"]:
        
        supprimer_dossiers([f"../images/screenshots/{zone['id']}/screenshots_{dimension}"])
        créer_dossiers([f"../données/{zone['id']}", f"../images/screenshots/{zone['id']}/screenshots_{dimension}",
            f"../images/screenshots/{zone['id']}/screenshots_{dimension}/retenus",
            f"../images/screenshots/{zone['id']}/screenshots_{dimension}/retenus_grand"])
        
        screenshot_rogné = couper_image(screenshot, screenshot.size[0], screenshot.size[1], dimension)
        for x in tqdm(range(0, screenshot_rogné.size[0], dimension), desc=f"- Dimension {dimension}px : ", ncols=100):
            for y in range(0, screenshot_rogné.size[1], dimension):
                
                ignorer = False
                for coordonnée_à_éviter in zones["coordonnées_à_éviter"]:
                    if (
                        x >= coordonnée_à_éviter["gauche"]
                        and x <= coordonnée_à_éviter["droite"]
                        and y >= coordonnée_à_éviter["haut"]
                        and y <= coordonnée_à_éviter["bas"]
                    ):
                        ignorer = True
                        break
                    
                if ignorer:
                    continue
                
                sous_screenshot = screenshot_rogné.crop((x, y, x + dimension, y + dimension))
                if not (est_obscure(sous_screenshot) or est_grise(sous_screenshot) or est_noire(sous_screenshot)):
                    sous_screenshot.save(f"../images/screenshots/{zone['id']}/screenshots_{dimension}/retenus/{x}_{y}_{x + dimension}_{y + dimension}.png")
                    
                    width, height = screenshot_rogné.size
                    
                    x1 = x + dimension / 2 - zone["dimensions"]["solution"] / 2
                    y1 = y + dimension / 2 - zone["dimensions"]["solution"] / 2
                    x2 = x + dimension / 2 + zone["dimensions"]["solution"] / 2
                    y2 = y + dimension / 2 + zone["dimensions"]["solution"] / 2
                    
                    fond = Image.new("RGB", (zone["dimensions"]["solution"], zone["dimensions"]["solution"]), (0, 0, 0))
                    
                    crop_x1 = max(0, x1)
                    crop_y1 = max(0, y1)
                    crop_x2 = min(width, x2)
                    crop_y2 = min(height, y2)
                    
                    sous_screenshot = screenshot_rogné.crop((crop_x1, crop_y1, crop_x2, crop_y2))
                    
                    paste_x = int(max(0, -x1))
                    paste_y = int(max(0, -y1))

                    fond.paste(sous_screenshot, (paste_x, paste_y))
                    fond.save(f"../images/screenshots/{zone['id']}/screenshots_{dimension}/retenus_grand/{x}_{y}_{x + dimension}_{y + dimension}.png")
                    fond.close()
                    
                    command = ["cwebp", "-q", str(80), "-mt", 
                        os.path.dirname(os.getcwd()).replace('\\', '/') + f"/images/screenshots/{zone['id']}/screenshots_{dimension}/retenus_grand/{x}_{y}_{x + dimension}_{y + dimension}.png",
                        "-o", os.path.dirname(os.getcwd()).replace('\\', '/') + f"/images/screenshots/{zone['id']}/screenshots_{dimension}/{x}_{y}_{x + dimension}_{y + dimension}.webp"
                    ]
                    subprocess.run(command, check=True)
                sous_screenshot.close()   
        screenshot_rogné.close()
        
        for fichier in lister_fichiers(f"../images/screenshots/{zone['id']}/screenshots_{dimension}/retenus"):
            dimensions = tuple([int(coordonnée) for coordonnée in fichier.split(".")[0].split("_")])
            coordonnées[fichier.split(".")[0] + ".webp"] = {
                "x": int((dimensions[0] + dimensions[2]) / 2 * (carte.size[0] / screenshot.size[0])), 
                "y": int((dimensions[1] + dimensions[3]) / 2 * (carte.size[1] / screenshot.size[1])),
            }
            
        supprimer_dossiers([f"../images/screenshots/{zone['id']}/screenshots_{dimension}/retenus_grand",
            f"../images/screenshots/{zone['id']}/screenshots_{dimension}/retenus"])

    sauvegarder_dictionnaire(f"../données/{zone['id']}/coordonnées.json", coordonnées)
    screenshot.close()
    carte.close()
    print()